# Automação Web e Busca de Informações com Python

Desafio:

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:

- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1QHOQt1J7d5Z4sBFKL9GN699OcPh2aVaP?usp=sharing

Para isso, vamos criar uma automação web:

Usaremos o selenium
Importante: baixar o webdriver

### Instalações:
- Selenium (automatiza o navegador por código) - Promp Anaconda "pip install selenium"
- Webdriver (Google Chrome - Chromedriver) - Coloque o arquivo na pasta do C:\Users\drumo\Anaconda3
- Webdriver (Firefox - Geckodriver)

- Baixar a versão correspondente ao seu Google Chrome (Ver Ajuda > Sobre o Google Chrome)

### Passo a passo:
- Passo 01: Pegar a cotação do Dólar
- Passo 02: Pegar a cotação do Euro
- Passo 03: Pegar a cotação do Ouro
- Passo 04: Importar a lista de produtos
- Passo 05: Recalcular o preço de cada produto
- Passo 06: Salvar os novos preços dos produtos

In [42]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Para rodar o código em 2 plano:
# from selenium.webdriver.chrome.options import Options
# chrome_options = Options()
# chrome_options.headless = True
# nav = webdriver.chrome(Options=chrome_options)

#Abrir um Navegador
navegador = webdriver.Chrome()
navegador.get("https://www.google.com/") #Acessa o site desejado

# Passo 01: Pegar a cotação do Dólar
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")
# find_element_by_xpath - Localiza o campo desejado (campo de busca)
# Código do Campo: Botão Direito > Inspecionar > Selecione 1 Ícone (setinha) > Selecione o campo desejado e clique
# Após o código ficar azul, clique com botão direito > Copy > Copy XPath
# send_keys - O que será inserido no campo de busca

navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
# send_keys - Aperta Enter

cotacao_dolar = navegador.find_element_by_xpath(
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

print(cotacao_dolar)

# -------------------------------------------------------------------
navegador.get("https://www.google.com/") #Acessa o site desejado

# Passo 02: Pegar a cotação do Euro
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
# find_element_by_xpath - Localiza o campo desejado (campo de busca)
# Código do Campo: Botão Direito > Inspecionar > Selecione 1 Ícone (setinha) > Selecione o campo desejado e clique
# Após o código ficar azul, clique com botão direito > Copy > Copy XPath
# send_keys - O que será inserido no campo de busca

navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
# send_keys - Aperta Enter

cotacao_euro = navegador.find_element_by_xpath(
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

print(cotacao_euro)

# -------------------------------------------------------------------
navegador.get("https://www.melhorcambio.com/ouro-hoje") #Acessa o site desejado

# Passo 03: Pegar a cotação do Ouro
navegador.find_element_by_xpath(
    '//*[@id="comercial"]').send_keys(Keys.ENTER)
# send_keys - Aperta Enter

cotacao_ouro = navegador.find_element_by_xpath(
    '//*[@id="comercial"]').get_attribute("value")

cotacao_ouro = cotacao_ouro.replace(",", ".")

print(cotacao_ouro)
# -------------------------------------------------------------------

navegador.quit()


5.1741
6.138707463
300.68


### Atualização da nossa base de preços com as novas cotações

Importando a base de dados

In [43]:
# Passo 04: Importar a lista de produtos
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)
print(tabela.info())

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Produtos             7 non-null      object 
 1   Preço Base Original  7 non-null      float64
 2   Moeda                7 non-null      object 
 3   Cotação              7 non-null      int64  
 4   Preço Base Reais     7 non-null      float64
 5   Ajuste               7 non-null      float64
 6   Preço Final          7 non-null      float64
dtypes: float64(4), int64(1), object(2)
memory usage: 520.0+ bytes
None


### Atualizando os preços e o cálculo do Preço Final

In [44]:
# Passo 05: Recalcular o preço de cada produto
# 1 - Atualizar a cotação
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# tabela.loc[linha, coluna] - Para selecionar um subconjunto de linhas e colunas
# float para garantir que seja passado o valor da forma correta

# 2 - Atualizar o preço base reais (preço base original * cotação)
tabela["Preço Base Reais"] = tabela["Preço Base Original"] * tabela["Cotação"]

# 3 - Atualizar o preço final (preço base reais * margem)
tabela["Preço Final"] = tabela["Preço Base Reais"] * tabela["Ajuste"]

display(tabela)


,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5.174100,5174.048259,1.40,7243.667563
1,Carro Renault,4500.00,Euro,6.138707,27624.183584,2.00,55248.367167
2,Notebook Dell,899.99,Dólar,5.174100,4656.638259,1.70,7916.285040
3,IPhone,799.00,Dólar,5.174100,4134.105900,1.70,7027.980030
4,Carro Fiat,3000.00,Euro,6.138707,18416.122389,1.90,34990.632539
5,Celular Xiaomi,480.48,Dólar,5.174100,2486.051568,2.00,4972.103136
6,Joia 20g,20.00,Ouro,300.680000,6013.600000,1.15,6915.640000


### Exportando a nova base de preços atualizada

In [49]:
# Passo 06: Salvar os novos preços dos produtos
tabela.to_excel("Produtos_Atualizado.xlsx", index=False)

# Para salvar por cima coloque o mesmo nome.